# QuakeFlow demo

In [1]:
import os
root_path = "./QuakeFlow/slurm/"
def run(root_path, script):
    cmd = f"cd {root_path} && python {script}"
    print(cmd)
    os.system(cmd)

### Set configuration parameters

**Edit the [config.json](./QuakeFlow/slurm/config.json) file to add parameters of studying region**

**Edit the [set_config.py](./QuakeFlow/slurm/set_config.py) file to select the studying region**

You can search for earthquake information here: [USGS](https://earthquake.usgs.gov/earthquakes/map/?extent=10.66061,-133.33008&extent=58.53959,-56.68945&range=month&magnitude=all&settings=true)

You can search for seismic network information here: [IRIS](http://ds.iris.edu/gmap/#network=*&starttime=2023-10-01&planet=earth)

In [2]:
run(root_path, "set_config.py")

cd ./QuakeFlow/slurm/ && python set_config.py
{
    "num_nodes": 1,
    "sampling_rate": 100,
    "degree2km": 111.1949,
    "channel": "HH*,BH*,EH*,HN*",
    "level": "response",
    "gamma": {
        "zmin_km": 0,
        "zmax_km": 60
    },
    "cctorch": {
        "sampling_rate": 100,
        "time_before": 0.25,
        "time_after": 1.0,
        "components": "ENZ123",
        "component_mapping": {
            "3": 0,
            "2": 1,
            "1": 2,
            "E": 0,
            "N": 1,
            "Z": 2
        }
    },
    "longitude0": -117.504,
    "latitude0": 35.705,
    "maxradius_degree": 0.5,
    "minlatitude": 35.205,
    "maxlatitude": 36.205,
    "minlongitude": -118.004,
    "maxlongitude": -117.004,
    "starttime": "2019-07-04T17:00:00",
    "endtime": "2019-07-04T19:00:00",
    "provider": [
        "SCEDC",
        "IRIS"
    ],
    "network": "CI"
}


### Download stations and events

**Edit the [download_station.py](./QuakeFlow/slurm/download_station.py) file to select the studying region**

**Edit the [download_catalog.py](./QuakeFlow/slurm/download_catalog.py) file to select the studying region**

In [3]:
run(root_path, "download_station.py")

run(root_path, "download_catalog.py")

cd ./QuakeFlow/slurm/ && python download_station.py
{
    "num_nodes": 1,
    "sampling_rate": 100,
    "degree2km": 111.1949,
    "channel": "HH*,BH*,EH*,HN*",
    "level": "response",
    "gamma": {
        "zmin_km": 0,
        "zmax_km": 60
    },
    "cctorch": {
        "sampling_rate": 100,
        "time_before": 0.25,
        "time_after": 1.0,
        "components": "ENZ123",
        "component_mapping": {
            "3": 0,
            "2": 1,
            "1": 2,
            "E": 0,
            "N": 1,
            "Z": 2
        }
    },
    "longitude0": -117.504,
    "latitude0": 35.705,
    "maxradius_degree": 0.5,
    "minlatitude": 35.205,
    "maxlatitude": 36.205,
    "minlongitude": -118.004,
    "maxlongitude": -117.004,
    "starttime": "2019-07-04T17:00:00",
    "endtime": "2019-07-04T19:00:00",
    "provider": [
        "SCEDC",
        "IRIS"
    ],
    "network": "CI"
}
Dowloaded 150 stations from scedc
Dowloaded 150 stations from iris
Parse 150 channels of SCED

/usr/local/python/3.10.8/lib/python3.10/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/python/3.10.8/lib/python3.10/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_ocean.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/python/3.10.8/lib/python3.10/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/python/3.10.8/lib/python3.10/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_lakes.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/python/3.10.8/lib/python3.10/site-packages

cd ./QuakeFlow/slurm/ && python download_catalog.py
{
    "num_nodes": 1,
    "sampling_rate": 100,
    "degree2km": 111.1949,
    "channel": "HH*,BH*,EH*,HN*",
    "level": "response",
    "gamma": {
        "zmin_km": 0,
        "zmax_km": 60
    },
    "cctorch": {
        "sampling_rate": 100,
        "time_before": 0.25,
        "time_after": 1.0,
        "components": "ENZ123",
        "component_mapping": {
            "3": 0,
            "2": 1,
            "1": 2,
            "E": 0,
            "N": 1,
            "Z": 2
        }
    },
    "longitude0": -117.504,
    "latitude0": 35.705,
    "maxradius_degree": 0.5,
    "minlatitude": 35.205,
    "maxlatitude": 36.205,
    "minlongitude": -118.004,
    "maxlongitude": -117.004,
    "starttime": "2019-07-04T17:00:00",
    "endtime": "2019-07-04T19:00:00",
    "provider": [
        "SCEDC",
        "IRIS"
    ],
    "network": "CI"
}
Dowloaded 252 events from scedc
Dowloaded 77 events from iris


### Download continuous waveform data

**Edit the [download_waveform_v2.py](./QuakeFlow/slurm/download_waveform_v2.py) file to select the studying region**

In [4]:
run(root_path, "download_waveform_v2.py")

cd ./QuakeFlow/slurm/ && python download_waveform_v2.py
Already downloaded from http://service.iris.edu: 2019-07-04T17:00:00 - 2019-07-04T18:00:00
Already downloaded from http://service.iris.edu: 2019-07-04T18:00:00 - 2019-07-04T19:00:00


### Run Phase Picking: PhaseNet

**Edit the [run_phasenet.py](./QuakeFlow/slurm/run_phasenet.py) file to select the studying region**

In [5]:
run(root_path, "run_phasenet.py")

cd ./QuakeFlow/slurm/ && python run_phasenet.py


2023-10-02 05:59:57.213849: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-02 05:59:57.241068: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 05:59:57.241107: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 05:59:57.241126: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 05:59:57.248165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-02 05:59:57.248364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Done with 8706 P-picks and 9200 S-picks


### Run Phase Association: GaMMA

**Edit the [run_gamma.py](./QuakeFlow/slurm/run_gamma.py) file to select the studying region**

In [2]:
run(root_path, "run_gamma.py")

cd ./QuakeFlow/slurm/ && python run_gamma.py
num_nodes: 1
sampling_rate: 100
degree2km: 111.1949
channel: HH*,BH*,EH*,HN*
level: response
gamma: {'zmin_km': 0, 'zmax_km': 60}
cctorch: {'sampling_rate': 100, 'time_before': 0.25, 'time_after': 1.0, 'components': 'ENZ123', 'component_mapping': {'3': 0, '2': 1, '1': 2, 'E': 0, 'N': 1, 'Z': 2}}
longitude0: -117.504
latitude0: 35.705
maxradius_degree: 0.5
minlatitude: 35.205
maxlatitude: 36.205
minlongitude: -118.004
maxlongitude: -117.004
starttime: 2019-07-04T17:00:00
endtime: 2019-07-04T19:00:00
provider: ['SCEDC', 'IRIS']
network: CI
use_dbscan: True
use_amplitude: True
method: BGMM
oversample_factor: 8
vel: {'p': 6.0, 's': 3.4285714285714284}
dims: ['x(km)', 'y(km)', 'z(km)']
x(km): [-45.14694189  45.14694189]
y(km): [-55.59745  55.59745]
z(km): (0, 20)
bfgs_bounds: ((-46.14694188923431, 46.14694188923431), (-56.59745, 56.59745), (0, 21), (None, None))
dbscan_eps: 10
dbscan_min_samples: 3
min_picks_per_eq: 5
min_p_picks_per_eq: 0
min_s_

### Visualization